In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import urdf2casadi.urdfparser as u2c
import numpy as np
import casadi as ca

In [2]:
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(
    project_root,
    'usage',
    'urdf',
    'alpha_5_robot_for_FD.urdf'
)
alpha.from_file(path_to_urdf)

root = "base_link"
tip = "alpha_standard_jaws_base_link"
gravity = [0, 0, -9.81]
n_joints = alpha.get_n_joints(root, tip)

qddot_sym = alpha.get_forward_dynamics_crba(root, tip)
# ─── 2. Define CasADi symbols and build the ODE ─────────────────────────────
q     = ca.MX.sym('q',     n_joints)
q_dot = ca.MX.sym('q_dot', n_joints)
tau   = ca.MX.sym('tau',   n_joints)
dt    = ca.MX.sym('dt')
gravity    = ca.MX.sym('gravity')

viscous = ca.MX.sym('viscous', n_joints)
coulomb = ca.MX.sym('coulomb', n_joints)

# compute joint accelerations
q_ddot = qddot_sym(q, q_dot, tau, gravity, viscous, coulomb)

# stack into first‐order form: x = [q; q_dot],  x_dot = [q_dot; q_ddot]
x    = ca.vertcat(q, q_dot)
x_dot = ca.vertcat(q_dot, q_ddot)*dt
p = ca.vertcat(dt,gravity,viscous, coulomb)
# ─── 3. Create an integrator ────────────────────────────────
dae  = {'x': x, 'p': p, 'u': tau, 'ode': x_dot}

intg = ca.integrator('intg', 'rk', dae, 0, 1, {
                    'simplify': True, 'number_of_finite_elements': 40})

res = intg(x0=x, u=tau, p=p)  # evaluate with symbols
x_next = res['xf']

F_next_ = ca.Function('Mnext', [x, tau, dt, gravity,viscous, coulomb], [x_next])

In [3]:
# c , cpp or matlab code generation for forward dynamics
F_next_.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

0